In [5]:
import neurokit2 as nk
import pandas as pd
import numpy as np

## Create polar_ecg_beats.csv

In [ ]:
import pandas as pd
import re

rows = []

with open("serial_output.txt", "r") as f:
    for line in f:
        line = line.strip()

        # Match timestamp and message
        match = re.match(r"\[(.*?)\]\s*>\s*(.*)", line)
        if not match:
            continue

        timestamp, message = match.groups()

        # Keep ONLY PolarRealtimeBPM events
        if not message.startswith("PolarRealtimeBPM"):
            continue

        # Split message
        event, values = message.split(":", 1)
        values = values.split(",")

        realtime_bpm = values[0]
        polar_bpm = values[1] if len(values) > 1 else None

        rows.append({
            "SystemTime": timestamp,
            "PolarRealtimeBPM": realtime_bpm,
            "PolarBPM": polar_bpm
        })

# Create DataFrame
df = pd.DataFrame(rows)

# Convert types
df["SystemTime"] = pd.to_datetime(df["SystemTime"])
df["PolarRealtimeBPM"] = pd.to_numeric(df["PolarRealtimeBPM"], errors="coerce")
df["PolarBPM"] = pd.to_numeric(df["PolarBPM"], errors="coerce")

# Save CSV
df.to_csv("polar_ecg_beats.csv", index=False)

print(f"Saved {len(df)} ECG beat events to polar_ecg_beats.csv")

Saved 366 ECG beat events to polar_ecg_beats.csv


## Calculate HRV

In [25]:
heartbeat_times = df["SystemTime"]
ibi_sec = heartbeat_times.diff().dt.total_seconds().dropna()

rmssd_ms = (np.sqrt(np.mean(np.diff(ibi_sec)**2)) * 1000)
print(f"RMSSD (ECG-derived): {rmssd_ms:.2f} ms")

RMSSD (ECG-derived): 121.77 ms


In [23]:
ibi_sec.describe()


count    365.000000
mean       0.808285
std        0.114928
min        0.369000
25%        0.736000
50%        0.802000
75%        0.870000
max        1.739000
Name: SystemTime, dtype: float64